# Using Zato from a notebook

So this is running in [Jupyter](https://jupyter.readthedocs.io/en/latest/install.html). 
I started down this route thinking of developer docs, but also realized that there's a use case here for potential customers. 

Notebooks could be a great way to allow tech users to interact with Blue Creek services and produce their own data analyses. 

We'd probably need to restrict outbound IPs to just those that are local to the Blue Creek production environment and/or whitelisted to prevent PHI disclosure, etc (e.g., ftp data out or pass to third party web services)

## setup for examples
you'll need to create a new channel for the admin.invoke service. 

you should be able to do that [here](http://localhost:9000/zato/http-soap/?cluster=1&connection=channel&transport=plain_http)

please set up like this:
![Channel Def](images/zato.admin.invoke.channel.png)

## example with zato.ping service

In [1]:
# example from zato.io
# if you get a connection error, make sure zato is running 
# by executing /vagrant/zatodev/go.sh
from zato.client import AnyServiceInvoker

address = 'http://localhost:11223'
path = '/zatodev/admin/invoke'
#auth = ('user for channel', 'password for channel')

client = AnyServiceInvoker(address, path, auth=None)
response = client.invoke('zato.ping')

if response.ok:
    print(response.data)
else:
    print(response.details)

{'pong': 'zato'}


# Example: Creating and loading a corpus

**I think this is busted because of the channel types and service handler interactions. e.g, handle_POST vs handle in the services to support restful naming conventions. I do wonder if it would be better to move all of blue creek into a library, but that gives issues to (e.g., how to call other services, etc). **

**Not sure what the right direction here is. Probably need to ask the folks on the zato mailing list.**

in this example, we just use our own service's name

In [ ]:
from zato.client import AnyServiceInvoker

address = 'http://localhost:11223'
path = '/zatodev/admin/invoke'
payload = {'name':'jupyter corpus', 'description':'sample description'}

client = AnyServiceInvoker(address, path, auth=None)
response = client.invoke('cte-corpora-addedit', payload=payload)

if response.ok:
    print(response.data)
else:
    print(response.details)


### workaround

In [10]:
# http://docs.python-requests.org/en/master/user/quickstart/
import requests
resp = requests.get('http://localhost:11223/corpora', data=None, headers={'Content-Type':'application/json'})

print resp.text
       

[K06KKCD4C7D1EVJZJ91WZ6Y6KNKH] Unknown URL:[/corpora] or SOAP action:[]


# Other things that you can do

### run shell commands


In [14]:
!tail -n 100 $ZATODIR/server1/logs/server.log

  File "/opt/zato/2.0.7/zato-server/src/zato/server/service/reqresp/__init__.py", line 343, in getvalue
    elem_value = self._getvalue(name, item, is_sa_namedtuple, is_required, leave_as_is)
  File "/opt/zato/2.0.7/zato-server/src/zato/server/service/reqresp/__init__.py", line 289, in _getvalue
    raise ZatoException(self.zato_cid, msg)
ZatoException: Expected elem:[status] not found in item:[{u'status': u'', u'data': u''}]

`
2016-06-18 20:47:17,345 - WARNING - 2004:Dummy-300 - zato.server.service:22 - Traceback (most recent call last):
  File "/opt/zato/2.0.7/zato-server/src/zato/server/service/__init__.py", line 405, in update_handle
    service.post_handle()
  File "/opt/zato/2.0.7/zato-server/src/zato/server/service/__init__.py", line 610, in post_handle
    resp = (self.response.payload.getvalue() if hasattr(self.response.payload, 'getvalue') else self.response.payload) or ''
  File "/opt/zato/2.0.7/zato-server/src/zato/server/service/reqresp/__init__.py", line 343, in getvalue